In [14]:
import pandas as pd
import numpy as np

# Thêm tiêu đề cột (dựa trên mô tả dữ liệu)
column_names = ['Name', 'Age', 'Weight', 'Sex', 'm0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']
df = pd.read_csv("patient_heart_rate.csv", names=column_names)

print(df.head())

   Name             Age  Weight         Sex m0006 m0612 m1218 f0006 f0612  \
0   1.0    Mickéy Mousé    56.0       70kgs    72    69    71     -     -   
1   2.0     Donald Duck    34.0   154.89lbs     -     -     -    85    84   
2   3.0      Mini Mouse    16.0         NaN     -     -     -    65    69   
3   4.0  Scrooge McDuck     NaN       78kgs    78    79    72     -     -   
4   5.0    Pink Panther    54.0  198.658lbs     -     -     -    69   NaN   

  f1218  
0     -  
1    76  
2    72  
3     -  
4    75  


In [18]:
# Chuyển đổi cột 'Name' sang kiểu chuỗi và xử lý NaN
df[['FirstName', 'LastName']] = df['Name'].str.split(expand=True)
df.drop('Name', axis=1)

print(df)

ValueError: Columns must be same length as key

In [19]:
def convert_weight_to_kg(weight):
    if 'lbs' in str(weight):
        return float(weight.replace('lbs', '')) * 0.453592
    elif 'kg' in str(weight):
        return float(weight.replace('kg', ''))
    else:
        return float(weight)

df['Weight'] = df['Weight'].apply(convert_weight_to_kg)

print(df['Weight'])

0     56.0
1     34.0
2     16.0
3      NaN
4     54.0
5     52.0
6     19.0
7     32.0
8      NaN
9      NaN
10    52.0
11    12.0
12     NaN
13    34.0
14     NaN
15     NaN
16    52.0
Name: Weight, dtype: float64


In [20]:
df.dropna(how='all', inplace=True)

print(df.isnull().sum())

Name      0
Age       3
Weight    6
Sex       6
m0006     2
m0612     2
m1218     2
f0006     2
f0612     3
f1218     2
dtype: int64


In [21]:
df.drop_duplicates(inplace=True)

print(df.duplicated().sum())

0


In [22]:
def remove_non_ascii(text):
    return ''.join(i if ord(i) < 128 else 'warning' for i in str(text))

df['FirstName'] = df['FirstName'].apply(remove_non_ascii)
df['LastName'] = df['LastName'].apply(remove_non_ascii)

print(df.head())

KeyError: 'FirstName'

In [23]:
# Thống kê dữ liệu thiếu
print("Dữ liệu thiếu Age:\n", df['Age'].isnull().sum())
print("Dữ liệu thiếu Weight:\n", df['Weight'].isnull().sum())

# Xử lý dữ liệu thiếu
age_mean = df['Age'].mean()
for index, row in df.iterrows():
    if pd.isnull(row['Age']) and pd.notnull(row['Weight']):
        df.loc[index, 'Age'] = age_mean
    elif pd.isnull(row['Weight']) and pd.notnull(row['Age']):
        df.loc[index, 'Weight'] = df['Weight'].mean()
    elif pd.isnull(row['Weight']) and pd.isnull(row['Age']):
        df.drop(index, inplace=True)
print(df.isnull().sum())

Dữ liệu thiếu Age:
 2
Dữ liệu thiếu Weight:
 5


TypeError: can only concatenate str (not "int") to str

In [24]:
def extract_pulse_rate(col_name, value):
    sex = col_name[0]
    time = col_name[1:]
    return pd.Series([value, sex, time])

new_cols = ['PulseRate', 'Sex_Time', 'Time']
new_rows = []
for col in ['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218']:
    new_rows.extend(df.apply(lambda row: extract_pulse_rate(col, row[col]), axis=1))

new_df = pd.DataFrame(new_rows, columns=new_cols)
df = pd.concat([df, new_df], axis=1)
df.drop(['m0006', 'm0612', 'm1218', 'f0006', 'f0612', 'f1218'], axis=1, inplace=True)

print(df.head())

ValueError: Shape of passed values is (18, 1), indices imply (18, 3)

In [25]:
def fill_missing_pulse_rate(row):
    if pd.isnull(row['PulseRate']):
        # Thay thế bằng giá trị trung bình liền trước và liền sau
        prev_value = df[(df['FirstName'] == row['FirstName']) & (df['Time'] < row['Time'])]['PulseRate'].iloc[-1] if not df[(df['FirstName'] == row['FirstName']) & (df['Time'] < row['Time'])].empty else np.nan
        next_value = df[(df['FirstName'] == row['FirstName']) & (df['Time'] > row['Time'])]['PulseRate'].iloc[0] if not df[(df['FirstName'] == row['FirstName']) & (df['Time'] > row['Time'])].empty else np.nan
        if pd.notnull(prev_value) and pd.notnull(next_value):
            return (prev_value + next_value) / 2
        # Thay thế bằng giá trị trung bình 2 giá liền trước
        prev_2_values = df[(df['FirstName'] == row['FirstName']) & (df['Time'] < row['Time'])].tail(2)['PulseRate'].mean()
        if pd.notnull(prev_2_values):
            return prev_2_values
        # Thay thế bằng giá trị trung bình 2 giá liền sau
        next_2_values = df[(df['FirstName'] == row['FirstName']) & (df['Time'] > row['Time'])].head(2)['PulseRate'].mean()
        if pd.notnull(next_2_values):
            return next_2_values
        # Thay thế bằng giá trị trung bình của người đó
        person_mean = df[df['FirstName'] == row['FirstName']]['PulseRate'].mean()
        if pd.notnull(person_mean):
            return person_mean
        # Thay thế bằng giá trị trung bình của nhóm giới tính
        gender_mean = df[df['Sex_Time'] == row['Sex_Time']]['PulseRate'].mean()
        if pd.notnull(gender_mean):
            return gender_mean
        # Thay thế bằng giá trị trung bình của toàn bộ dữ liệu
        data_mean = df['PulseRate'].mean()
        if pd.notnull(data_mean):
            return data_mean
        # Thay thế bằng mức ổn định trong y học (ví dụ: 80)
        return 80
    return row['PulseRate']

df['PulseRate'] = df.apply(fill_missing_pulse_rate, axis=1)

print(df['PulseRate'])

KeyError: 'PulseRate'

In [26]:
df = df[['FirstName', 'LastName', 'Age', 'Weight','Sex_Time','Time','PulseRate']]
df.reset_index(drop=True, inplace=True)
df.to_csv('patient_heart_rate_clean.csv', index=False)

KeyError: "['FirstName', 'LastName', 'Sex_Time', 'Time', 'PulseRate'] not in index"